In [2]:
import ldap3

In [38]:
# import class and constants
from ldap3 import Server, Connection, ALL

username = 'CN=path-web-auth-int,CN=users,DC=umhs,DC=med,DC=umich,DC=edu'
password = "bk62pq"
ldap_server = "umhs.med.umich.edu"

# define the server
s = Server(ldap_server, port=389, get_info=ALL)  # define an unsecure LDAP server, requesting info on DSE and schema

# define the connection
c = Connection(s, user=username, password=password)

# perform the Bind operation
print(c.bind())
if not c.bind():
    print(c.result['description'])

True


In [36]:
# import class and constants
from ldap3 import Server, Connection, ALL

def ldap_authenticate(user, password):
    username = 'CN=' + user + ',CN=users,DC=umhs,DC=med,DC=umich,DC=edu'
    print(username)
    ldap_server = "umhs.med.umich.edu"

    # define the server
    s = Server(ldap_server, port=389, get_info=ALL)  # define an unsecure LDAP server, requesting info on DSE and schema

    # define the connection
    c = Connection(s, user=username, password=password)

    # perform the Bind operation
    return(c.bind())